In [1]:
import datetime
today=f"{datetime.datetime.now():%Y-%m-%d_%H-%M}"
today

'2022-03-01_18-16'

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
#imports
import pandas as pd
import pickle as pickle
from scipy.spatial.distance import cdist, pdist, squareform
#import backspinpy
import pandas as pd
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import StratifiedShuffleSplit  
from collections import defaultdict
from sklearn import preprocessing
import matplotlib.patches as mpatches
from numba import jit
#from .computeElasticPrincipalGraph import computeElasticPrincipalGraph

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [9]:
cd D:\Dropbox\data\proj\PE_HYZ\PublicDataSet\GBM_Couturier2020_DevGBM

D:\Dropbox\data\proj\PE_HYZ\PublicDataSet\GBM_Couturier2020_DevGBM


In [10]:
import loompy

In [11]:
ds=loompy.connect("2021-08-16_17-03_NewRef_Couturier2020_DevGBM_Vs_SL_devMouseBrain_normalizedExprMatrix.loom")

In [14]:
df=pd.DataFrame(ds[:,:])
df.columns=ds.ca['CellID']
df.index=ds.ra['Gene']
df.shape

(1943, 50304)

In [18]:
cd D:\Dropbox\data\proj\PE_HYZ\PublicDataSet\GBM_Couturier2020_DevGBM

D:\Dropbox\data\proj\PE_HYZ\PublicDataSet\GBM_Couturier2020_DevGBM


In [19]:
ds=loompy.connect("20210809FinalMatrixWithCluster.loom")

In [20]:
ds.shape

(33660, 18475)

In [21]:
dfgbm=pd.DataFrame(ds[:,:])
dfgbm.index=ds.ra["Gene"]
dfgbm.columns=ds.ca['CellID']
#dfgbm.loc["PatientID"]=ds.ca['PatientID']
dfgbm.loc["Cluster"]=ds.ca['Cluster']

In [25]:
dfgbm.shape

(33661, 18475)

In [28]:
dfclus=dfgbm.iloc[-2:,:]
dfgbm=dfgbm.iloc[:-1,:].astype(float)

In [32]:
dfgbmclus=dfclus.iloc[-1:,:]
##dfgbm=dfgbm.iloc[:-1,:]
dfgbmclus

,N10_TACGGATCACCAACCG-1,N10_TACGGATCACCCAGTG-1,N10_TCAGCTCAGAGCTGCA-1,N10_CGTTGGGGTCAACATC-1,N10_AACCGCGGTACTTCTT-1,N10_ATAGACCGTCAAAGCG-1,N10_AAACCTGCACGAGAGT-1,N10_TATTACCAGAGACGAA-1,N10_GAATGAATCGAGAACG-1,N10_GTAGGCCCATAGGATA-1,...,N9_ATAAGAGTCCGTAGTA-1,N9_CCTTACGTCAGCGATT-1,N9_GAATAAGCACGGTGTC-1,N9_ACGCCAGGTAGGGACT-1,N9_CCTACACTCTGCCAGG-1,N9_TGTATTCGTAGAAAGG-1,N9_CGTTCTGAGTTACGGG-1,N9_CGTAGCGCAAGGTGTG-1,N9_GTTCATTGTCCCTTGT-1,N9_AACTGGTCACATAACC-1
Cluster,Neuronal,Neuronal,Oligo,Progenitor,Unassigned,Neuronal,Neuronal,Progenitor,Unassigned,Progenitor,...,Astro,Astro,Mesenchymal,Unassigned,Mesenchymal,Unassigned,Mesenchymal,Mesenchymal,Unassigned,Mesenchymal


In [33]:
cd d:/Dropbox/data/proj/PE_HYZ/PublicDataSet/Developmental_mouse brain

d:\Dropbox\data\proj\PE_HYZ\PublicDataSet\Developmental_mouse brain


In [34]:
dfposi=pd.read_table("20201111_MouseBrainAtlas_SL_metaInfo.tab",
                 index_col=0, header=0, sep="\t")

In [35]:
dfposi.head()

,Cluster,ClusterName,Region,PseudoAge,TSNE_X,TSNE_Y,UMAP_X,UMAP_Y,BTSNE_X,BTSNE_Y,UMAP3D_X,UMAP3D_Y,UMAP3D_Z
10X25_3_A_1:TTGGTACTGGGACAx,Neural_crest,634,Midbrain,9.505,-25.900671,-6.408116,6.208536,7.370287,-56.705326,-15.175609,2.873503,3.853158,-6.144522
10X27_3_A_1:TATCAGCTCTTGCCx,Neural_crest,634,Midbrain,9.540,-26.354128,-1.820086,5.459467,8.159930,-55.292125,-10.095811,1.731046,3.764495,-6.431027
10X27_3_A_1:AACCAGTGCTCAGAx,Neural_crest,634,Midbrain,9.930,-26.986095,-7.087025,6.572917,7.498238,-61.365980,-15.862534,3.126918,3.940230,-6.405674
10X27_3_A_1:CTAGTTTGCCTGAAx,Neural_crest,634,Midbrain,10.230,-27.583021,-6.904059,6.436008,7.521403,-62.872862,-14.369149,2.961187,4.019217,-6.277933
10X27_3_A_1:GTAGACTGGGATCTx,Neural_crest,634,Midbrain,10.635,-26.837190,-8.949903,6.671855,7.109936,-64.516093,-16.561974,3.474845,3.712845,-6.309426


In [36]:
dfpfcclus=dfposi.T.loc[["Cluster","TSNE_X","TSNE_Y"]]
dfpfcclus.shape

(3, 127326)

In [37]:
set(dfposi)

{'BTSNE_X',
 'BTSNE_Y',
 'Cluster',
 'ClusterName',
 'PseudoAge',
 'Region',
 'TSNE_X',
 'TSNE_Y',
 'UMAP3D_X',
 'UMAP3D_Y',
 'UMAP3D_Z',
 'UMAP_X',
 'UMAP_Y'}

In [38]:
proto = pd.Series({'Choroid plexus':'Choroid plexus',
 'Early Neural Tube': 'Neural_tube',
                    'Neural_tube': 'Neural_tube',
                   'Neural Tube': 'NT',
 'Ectoderm':'Ectoderm',
 'Fibroblast':'VLMC',
                   'VLMC':'VLMC',
                    'NT/Rgl': 'NT/Rgl',
 'Glia': 'Glia',
 'Neural crest': 'Neural_crest',
                    'Neural_crest': 'Neural_crest',
 'Neuron':'Neuron',
 'OPCs': 'OPCs',
 'Radial glia':  'Rgl',
                   'Rgl':'Rgl',
 'Vascular':'Pericyte/SMC',
                   'PeriV_Mural':'Pericyte/SMC',
                   'Pericyte/SMC':'Pericyte/SMC',
       })


posict_dev = dfposi['Cluster'].values
posiprotogruop = proto.loc[posict_dev].values
len(posiprotogruop)

127326

In [39]:
set(posiprotogruop)

{'Choroid plexus',
 'Ectoderm',
 'Glia',
 'Neural_crest',
 'Neural_tube',
 'Neuron',
 'OPCs',
 'Pericyte/SMC',
 'Rgl',
 'VLMC'}

In [40]:
dfposi['Cluster']=posiprotogruop

In [41]:
dfposi.head()

,Cluster,ClusterName,Region,PseudoAge,TSNE_X,TSNE_Y,UMAP_X,UMAP_Y,BTSNE_X,BTSNE_Y,UMAP3D_X,UMAP3D_Y,UMAP3D_Z
10X25_3_A_1:TTGGTACTGGGACAx,Neural_crest,634,Midbrain,9.505,-25.900671,-6.408116,6.208536,7.370287,-56.705326,-15.175609,2.873503,3.853158,-6.144522
10X27_3_A_1:TATCAGCTCTTGCCx,Neural_crest,634,Midbrain,9.540,-26.354128,-1.820086,5.459467,8.159930,-55.292125,-10.095811,1.731046,3.764495,-6.431027
10X27_3_A_1:AACCAGTGCTCAGAx,Neural_crest,634,Midbrain,9.930,-26.986095,-7.087025,6.572917,7.498238,-61.365980,-15.862534,3.126918,3.940230,-6.405674
10X27_3_A_1:CTAGTTTGCCTGAAx,Neural_crest,634,Midbrain,10.230,-27.583021,-6.904059,6.436008,7.521403,-62.872862,-14.369149,2.961187,4.019217,-6.277933
10X27_3_A_1:GTAGACTGGGATCTx,Neural_crest,634,Midbrain,10.635,-26.837190,-8.949903,6.671855,7.109936,-64.516093,-16.561974,3.474845,3.712845,-6.309426


In [43]:
dfnn =df.T
dfnn.shape

(50304, 1943)

In [44]:
dfnn.head()

,SPON2,IL1R1,PLP1,NEUROD6,DDA1,OCLN,CAPN5,HESX1,NRN1,TP53I11,...,CHRD,CLDN3,LMCD1,NRXN1,TBX18,LBP,PMM1,TUBB3,COL1A2,HTR2C
10X73_6_A_1:CTTGTATGGCTAACx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
10X18_2_A_1:GATAAGGAAGGTCTx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10X18_2_A_1:TATACAGATGTCGAx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
10X73_6_A_1:AGAGCGGAACTGTGx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10X70_6_A_1:ATACTCTGCAGTTGx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [47]:
dfposi=dfposi.loc[set(dfposi.index)&set(dfnn.index)]

,Cluster,ClusterName,Region,PseudoAge,TSNE_X,TSNE_Y,UMAP_X,UMAP_Y,BTSNE_X,BTSNE_Y,UMAP3D_X,UMAP3D_Y,UMAP3D_Z
10X16_5_A_1:GGAGTTTGAACGAAx,Neuron,38,Forebrain,15.695,10.401540,12.368582,-1.608322,-7.450790,14.887502,42.255999,-0.101194,-3.553880,4.312026
10X08_1_A_1:ACAGTGTGTTGTGGx,Neural_crest,650,Head,8.725,-22.106678,-9.627563,5.244210,6.761459,-54.167228,-21.865280,2.654018,2.900583,-5.554272
10X17_2_A_1:GCCGACGAATTGGCx,VLMC,578,Hindbrain,16.445,-25.428677,-18.448587,9.720346,7.144159,-85.593157,1.387060,3.787744,1.852242,-9.103201
10X72_3_A_1:CGTAGCCTGGTGGAx,Neural_crest,711,Forebrain,9.345,-15.937236,-9.092426,4.099833,6.358791,-35.699129,-23.801537,2.002173,2.415203,-4.660997
10X30_5_A_1:GAAACCTGAGAGTAx,Neuron,407,Forebrain,15.450,4.166421,-16.023070,-6.166296,1.564187,16.886936,-29.986398,-1.476650,0.704077,3.920854


In [58]:
dfposi=dfposi[["TSNE_X","TSNE_Y","Cluster"]]
dfposi.columns=['UMAP1', 'UMAP2', 'Cluster']
dfposi["Index"]=range(len(dfposi.index))

In [59]:
proto = pd.Series({'Choroid plexus':'Choroid plexus',
 'Early Neural Tube': 'Neural_tube',
                    'Neural_tube': 'Neural_tube',
                   'Neural Tube': 'NT',
 'Ectoderm':'Ectoderm',
 'Fibroblast':'VLMC',
                   'VLMC':'VLMC',
                    'NT/Rgl': 'NT/Rgl',
 'Glia': 'Glia',
 'Neural crest': 'Neural_crest',
                    'Neural_crest': 'Neural_crest',
 'Neuron':'Neuron',
 'OPCs': 'OPCs',
 'Radial glia':  'Rgl',
                   'Rgl':'Rgl',
 'Vascular':'Pericyte/SMC',
                   'PeriV_Mural':'Pericyte/SMC',
                   'Pericyte/SMC':'Pericyte/SMC',
       })
mct_dev = dfposi['Cluster'].values
mprotogruop = proto.loc[mct_dev].values
len(mprotogruop)

31795

In [60]:
dfposi['Cluster']=mprotogruop

In [61]:
set(mprotogruop)

{'Choroid plexus',
 'Ectoderm',
 'Glia',
 'Neural_crest',
 'Neural_tube',
 'Neuron',
 'OPCs',
 'Pericyte/SMC',
 'Rgl',
 'VLMC'}

## Preprocessing and Visualization

In [101]:
cd D:\Dropbox\data\proj\PE_HYZ\BuidingCode\ready_SWAPLINE

D:\Dropbox\data\proj\PE_HYZ\BuidingCode\ready_SWAPLINE


In [63]:
import SWAPLINEmain

In [64]:
dfcellclusumap1, dfcellclusumap2=SWAPLINEmain.SWAPLINE_dist(n_pcs=30,dfnn=dfnn, n_neighbors=len(dfnn.index),  metric='euclidean', dfposi=dfposi)

C:\Users\huyiz\AppData\Local\Continuum\anaconda3\envs\py38\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str_']. An error will be raised in 1.2.
  warnings.warn(


[Processing-0%--0%--0%--0%--0%--0%--0%--0%--0%--0%-]


In [65]:
dfcellclusumap1.to_csv("%s_50270Final_Couturier2020_DevGBM_NewRef_NN20_rd_dfcellclusumap1.tab"%today,sep="\t")

In [66]:
dfcellclusumap2.to_csv("%s_50270Final_Couturier2020_DevGBM_NewRef_NN20_rd_dfcellclusumap2.tab"%today,sep="\t")

In [103]:
cd D:\Dropbox\data\proj\PE_HYZ\PublicDataSet\GBM_Couturier2020_DevGBM

D:\Dropbox\data\proj\PE_HYZ\PublicDataSet\GBM_Couturier2020_DevGBM


In [104]:
dfprobCL=pd.read_table("2021-08-16_17-03_Probability_Reliable_Couturier2020_DevGBM_Vs_SL_devMouseBrain_Reference_count.tab",index_col=0,header=0,sep="\t")
dfprobCL.shape

(18475, 10)

In [106]:
negtest=dfpermutation.values
negtest

[8.286483,
 9.999207,
 16.809064,
 4.751205,
 11.166089,
 2.439122,
 9.515249,
 6.65107,
 10.298396,
 9.834658]

In [107]:
n= len(set(dfposi["Cluster"]))
n

10

In [123]:
nodelist=[['Neural_crest', 'Neural_tube',  'Ectoderm'],['Neural_crest','Pericyte/SMC',  'VLMC'],['Neural_crest', 'Ectoderm','VLMC'],
['Rgl','Neural_tube',  'Ectoderm'],['Rgl','Neural_tube',  'Glia'],['Rgl','Neural_tube',  'OPCs'],['Rgl','Neural_tube',  'Neuron'],
['Rgl','OPCs',  'Neuron'],['Rgl','Glia', 'Neuron'],['Rgl','Glia', 'OPCs']]

In [127]:
cd D:\Dropbox\data\proj\PE_HYZ\BuidingCode\ready_SWAPLINE

D:\Dropbox\data\proj\PE_HYZ\BuidingCode\ready_SWAPLINE


In [134]:
import SWAPLINEmainR6

In [135]:
dfnewumapX=SWAPLINEmainR6.SWAPLINE_assign(dfprobCL=dfprobCL, dfcellclusumap1=dfcellclusumap1,dfcellclusumap2=dfcellclusumap2,
                                          negtest=negtest, n=len(set(dfposi["Cluster"])), nodelist=nodelist)

In [137]:
#plot...